In [1]:
import pickle 
import numpy as np 
import pandas as pd
import networkx as nx
from tqdm import tqdm
import time
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import collections

from src.utils.data_utils import *
from src.utils.general_utils import *

C:\Users\pierr\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
movie_path = 'data/Movie.pkl'
actors_path = 'data/Actor.pkl'
actors_career_path = 'data/Actor_career.pkl'

with open(movie_path, 'rb') as file:
    Movie = pickle.load(file)

with open(actors_path, 'rb') as file:
    Actor = pickle.load(file)

with open(actors_career_path, 'rb') as file:
    Actor_career = pickle.load(file)

C:\Users\pierr\AppData\Local\Temp\ipykernel_7236\2221900138.py:6: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  Movie = pickle.load(file)


In [235]:
Actor["actor_age_atmovierelease"] = Actor["actor_age_atmovierelease"].apply(
    lambda x: [pd.NA if pd.isna(age) else int(age) for age in x]
)

Actor["actor_age_atmovierelease"] = Actor["actor_age_atmovierelease"].apply(
    lambda x: [-1 if (pd.isna(val) or val < 0) else val for val in x]
)

Actor["Career_Start_age"] = Actor.apply(
    lambda row: min([val for val in row["actor_age_atmovierelease"] if val > 0], default=-1),
    axis=1,
)


max_career_years = int(max(Actor.apply(lambda row: max(row["actor_age_atmovierelease"]),axis=1)-Actor["Career_Start_age"]))

Nbr_films = [f"Nbr_films_{i+1}" for i in range(100)]
Degree = [f"Degree_{i+1}" for i in range(100)]
Centrality = [f"Centrality_{i+1}" for i in range(100)]


Actor_career = pd.concat(
    [Actor.copy(), pd.DataFrame(0.0, index=Actor.index, columns=Nbr_films), pd.DataFrame(0.0, index=Actor.index, columns=Degree), pd.DataFrame(0.0, index=Actor.index, columns=Centrality)], axis=1)

In [237]:
def fill_career_years(row):

    age_at_release = np.array(row["actor_age_atmovierelease"])
    counts = np.zeros(max_career_years+1)
    
    if len(age_at_release) != 0:
        for year in age_at_release:
            if year >= 0 and row["Career_Start_age"]>=0:
                counts[year-row["Career_Start_age"]] += 1
                    
    return counts

In [238]:
start = time.time()

movie_releasedates = range(1920,2020) 
Actor_career['carrer_year'] = 0
career_counts = np.array(Actor_career.apply(fill_career_years, axis=1).tolist())

for i in range(max_career_years):
    Actor_career[f"Year_{i+1}"] = career_counts[:, i]
    Actor_career[f"Nbr_films_{i+1}"] = career_counts[:, i]


for date in movie_releasedates:

    played_together = create_actor_network(Actor_career, Movie, 1, min_releasedate=date)

    print(f"Network ({date+1}/2024) of {played_together.number_of_nodes()} nodes.")

    nan_nodes = [node for node in played_together.nodes if pd.isna(node)]
    played_together.remove_nodes_from(nan_nodes)

    degrees = dict(played_together.degree())
    eigenvector_centrality = nx.eigenvector_centrality(played_together, max_iter=1000)
    eigenvector_values = np.array(list(eigenvector_centrality.values()))

    mean = np.mean(eigenvector_values)
    std = np.std(eigenvector_values)
    standardized_eigenvector_centrality = {node: (val - mean) / std for node, val in eigenvector_centrality.items()}


    actor_id_to_idx = pd.Series(Actor_career.index, index=Actor_career["Freebase_actor_ID"]).to_dict()
    num_actors = len(Actor_career)
    degree_updates = np.zeros(num_actors, dtype=np.float32)
    centrality_updates = np.zeros(num_actors, dtype=np.float32)
    career_year_updates = np.zeros(num_actors, dtype=np.int32)


    for actor_id in tqdm(played_together.nodes(), desc="Fill the dataset"):
        if actor_id in actor_id_to_idx:
            actor_idx = actor_id_to_idx[actor_id]
            career_year = Actor_career.at[actor_idx, "carrer_year"]
            
            degree_updates[actor_idx] = degrees.get(actor_id, 0)
            centrality_updates[actor_idx] = standardized_eigenvector_centrality.get(actor_id, 0.0)
            career_year_updates[actor_idx] = career_year

            Actor_career.at[actor_idx, "carrer_year"] += 1


    for actor_idx, career_year in enumerate(career_year_updates):
        if career_year > 0:  # Only update valid career years
            Actor_career.at[actor_idx, f"Degree_{career_year}"] = degree_updates[actor_idx]
            Actor_career.at[actor_idx, f"Centrality_{career_year}"] = centrality_updates[actor_idx]
    
print(f"Computation time: {time.time()-start:.2f}s")



C:\Users\pierr\AppData\Local\Temp\ipykernel_10352\2840603769.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Actor_career[f"Year_{i+1}"] = career_counts[:, i]
C:\Users\pierr\AppData\Local\Temp\ipykernel_10352\2840603769.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Actor_career[f"Year_{i+1}"] = career_counts[:, i]
C:\Users\pierr\AppData\Local\Temp\ipykernel_10352\2840603769.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

Network (1907/2024) of 11 nodes.


Creating network: 100%|██████████| 64308/64308 [00:04<00:00, 13625.15it/s]


Network (1908/2024) of 15 nodes.


Creating network: 100%|██████████| 64308/64308 [00:04<00:00, 15566.69it/s]


...

Network (2019/2024) of 119364 nodes.


Fill the dataset: 100%|██████████| 119364/119364 [00:16<00:00, 7196.23it/s]
C:\Users\pierr\AppData\Local\Temp\ipykernel_10352\2840603769.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Actor_career.at[actor_idx, f"Degree_{career_year}"] = degree_updates[actor_idx]
C:\Users\pierr\AppData\Local\Temp\ipykernel_10352\2840603769.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Actor_career.at[actor_idx, f"Centrality_{career_year}"] = centrality_updates[actor_idx]
Creating network: 100%|██████████| 64308/64308 [00:21<00:00, 2

Network (2020/2024) of 119365 nodes.


Fill the dataset: 100%|██████████| 119365/119365 [00:19<00:00, 6170.42it/s]
C:\Users\pierr\AppData\Local\Temp\ipykernel_10352\2840603769.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Actor_career.at[actor_idx, f"Degree_{career_year}"] = degree_updates[actor_idx]
C:\Users\pierr\AppData\Local\Temp\ipykernel_10352\2840603769.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Actor_career.at[actor_idx, f"Centrality_{career_year}"] = centrality_updates[actor_idx]


Computation time: 4317.73s


In [5]:
Actor_career

,Freebase_actor_ID,actor_name,actor_DOB,actor_gender,actor_height,ethnicity,Freebase_movie_ID,actor_age_atmovierelease,Career_Start_age,Nbr_films_1,...,Degree_109,Centrality_109,Degree_110,Centrality_110,Degree_111,Centrality_111,Degree_112,Centrality_112,Degree_113,Centrality_113
0,/m/010p3,Adam Carolla,1964-05-27,M,1.88,Italian Americans,"[/m/0bq2wj, /m/08hjr9, /m/06zm9p7, /m/06c5z_, ...","[35, 35, 45, 33, 33, 34, 42, 35, -1, 44, 47, 38]",33,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/m/010q36,Fred Rogers,1928-03-20,M,NaN,None,[/m/033pf1],[67],67,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/m/010wx,Aria Giovanni,1977-11-03,F,1.68,Yugoslavs,[/m/07gx0c],[25],25,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/m/010xjr,Richard Harris,1930-10-01,M,1.85,Irish people,"[/m/0c_6bf, /m/078mm1, /m/03qfyt, /m/0cz7_4w, ...","[66, 46, -1, 59, 48, 39, 66, 50, 71, 71, 68, 7...",28,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/m/0112yl,Toshiro Mifune,1920-04-01,M,1.74,None,"[/m/0513nd, /m/05c2g3q, /m/04wjg1, /m/01tch8, ...","[31, 42, 45, 41, 62, 29, 30, 38, 35, 36, 59, 6...",28,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135755,/m/0z58,Ardal O'Hanlon,1965-10-08,M,NaN,None,[/m/027cvd8],[-1],-1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135756,/m/0z9q0,"José Luis Rodríguez ""El Puma""",1943-01-14,M,NaN,None,[/m/0gls5qd],[69],69,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135757,/m/0zcb7,Marcus Giamatti,1961-10-03,M,1.85,None,"[/m/080kkcx, /m/09v460, /m/02drs7, /m/0gls5v_,...","[38, 29, 40, 48, 29, 46]",29,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135758,/m/0zcbl,Paul Giamatti,1967-06-06,M,1.74,Irish Americans,"[/m/0ck5rr, /m/04yg13l, /m/04wdfw, /m/03whlgj,...","[38, 43, 28, 41, 46, 43, 28, 31, 30, 37, 24, 3...",24,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
Actor_career['Degree_113'].max()

np.float32(10.0)

In [260]:
Actor_career.to_pickle('data/Actor_career.pkl')